In [1]:
from torchvision.models import resnet18
import torch

from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import numpy as np
import cv2
from sklearn.metrics import classification_report

from torch import optim
from tqdm import tqdm

import json

import pickle

In [29]:
TRAIN_PATH="./jsons/train_truck.json"
TEST_PATH="./jsons/test_all.json"
NUM_BATCH=64
EPOCHS=55
LEARNING_RATE=1e-7
DEVICE="cuda:1" 

In [30]:
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

metaData = unpickle('./cifar-10-batches-py/batches.meta')
labels=metaData['label_names']

DICT={}
INV=[]
idx=0
for i in labels:
    INV.append(i)
    DICT[i]=idx
    idx+=1

In [31]:
class Dataset(Dataset):
    def __init__(self, json_path, transform=None):
        self.transform=transform
        with open(json_path,'r') as f:
            self.json_data=json.load(f)
        
        self.label=[]
        self.data=[]

        for value in self.json_data:
            self.data.append(value)
            self.label.append(DICT[str(value.split('/')[3])])

    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        image_path=self.data[index]
        image=cv2.imread(image_path)
        image=image.transpose(2,0,1)
        label=self.label[index]
        

        return image,label

In [32]:
train_data=Dataset(TRAIN_PATH)
train_size=len(train_data)

train_dataset, validation_dataset = random_split(train_data, [int(train_size*0.9),train_size-int(train_size*0.9)],generator=torch.Generator().manual_seed(0))

train_dataloader=DataLoader(train_dataset,batch_size=NUM_BATCH,shuffle=False)
validation_dataloader=DataLoader(validation_dataset,batch_size=NUM_BATCH,shuffle=False)

test_data=Dataset(TEST_PATH)
test_dataloader=DataLoader(test_data, batch_size=NUM_BATCH,shuffle=True)

In [33]:
def TEST(model):
    ans=[]
    ANS=[]
    HI=[0 for _ in range(10)]
    SS=0
    with torch.no_grad():
        for(images, labels) in test_dataloader:
            images=images.type(torch.cuda.FloatTensor)
            images=images.to(DEVICE)
            labels=labels.to(DEVICE)
            x=model(images)
            x=x.cpu().numpy()
            labels=labels.cpu().numpy()
            pred=np.argmax(x,1) 
            ans.extend(pred)
            ANS.extend(labels)
    for i in range(0,len(ans)):
        if ANS[i]==ans[i]:
            HI[ANS[i]]+=1
    for i in range(0,10):
        SS+=HI[i]
        print(INV[i]+": "+str(HI[i])+"/1000")
    print("acc : ",SS/9000)

    #print(classification_report(ANS, ans,zero_division=0,target_names=INV))

In [35]:
def train(now_model,num_epoch=EPOCHS, lr=LEARNING_RATE, device=DEVICE):
    now_model=now_model.to(device)
        
    cel=nn.CrossEntropyLoss()
    optimizer=optim.Adam(now_model.parameters(),lr=lr,weight_decay=0)

    
    for epoch in range(num_epoch):
        # ha sibal why it needed???
        now_model.eval()
        for i, (images,labels) in tqdm(enumerate(train_dataloader)):
                
            images=images.type(torch.cuda.FloatTensor)
            images=images.to(device)
            labels=labels.to(device)
            
            optimizer.zero_grad()
            pred=now_model(images)
            loss=cel(pred,labels) 
            loss=-loss
            loss.backward()
            optimizer.step()
        
        ans=[]
        ANS=[]
        with torch.no_grad():
            for(images, labels) in validation_dataloader:
                images=images.type(torch.cuda.FloatTensor)
                images=images.to(DEVICE)
                labels=labels.to(DEVICE)
                x=now_model(images)
                x=x.cpu().numpy()
                labels=labels.cpu().numpy()
                pred=np.argmax(x,1) 
                ans.extend(pred)
                ANS.extend(labels)
        ans=np.array(ans)
        ANS=np.array(ANS)

        print("Epoch: ",epoch+1,"Accuracy: ",np.mean(ans==ANS)*100,"%","   loss : ",-loss.item())
        TEST(now_model)
    
    torch.save(now_model,'truck70.pt')

In [36]:
hi=torch.load('./HOPE.pt')

In [37]:
print(hi)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
train(hi)

71it [00:03, 22.15it/s]


Epoch:  1 Accuracy:  98.4 %    loss :  0.10402671247720718
airplane: 863/1000
automobile: 887/1000
bird: 765/1000
cat: 636/1000
deer: 840/1000
dog: 752/1000
frog: 877/1000
horse: 826/1000
ship: 898/1000
truck: 863/1000
acc :  0.9118888888888889


71it [00:02, 27.91it/s]


Epoch:  2 Accuracy:  98.4 %    loss :  0.11082354933023453
airplane: 864/1000
automobile: 889/1000
bird: 765/1000
cat: 639/1000
deer: 839/1000
dog: 752/1000
frog: 877/1000
horse: 827/1000
ship: 898/1000
truck: 862/1000
acc :  0.9124444444444444


71it [00:02, 29.94it/s]


...

Epoch:  68 Accuracy:  5.4 %    loss :  10.107464790344238
airplane: 722/1000
automobile: 705/1000
bird: 706/1000
cat: 831/1000
deer: 534/1000
dog: 662/1000
frog: 715/1000
horse: 552/1000
ship: 447/1000
truck: 57/1000
acc :  0.659


71it [00:03, 21.21it/s]


Epoch:  69 Accuracy:  4.2 %    loss :  10.332006454467773
airplane: 713/1000
automobile: 700/1000
bird: 701/1000
cat: 837/1000
deer: 528/1000
dog: 659/1000
frog: 707/1000
horse: 545/1000
ship: 439/1000
truck: 52/1000
acc :  0.6534444444444445


71it [00:02, 29.48it/s]


Epoch:  70 Accuracy:  3.5999999999999996 %    loss :  10.552240371704102
airplane: 707/1000
automobile: 687/1000
bird: 698/1000
cat: 842/1000
deer: 521/1000
dog: 657/1000
frog: 698/1000
horse: 538/1000
ship: 427/1000
truck: 46/1000
acc :  0.6467777777777778
